# Verwendete Bibliotheken & Initialisierung

In [ ]:
# Falls nötig:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# pip install -U "transformers[torch]"

In [ ]:
from langchain_text_splitters  import RecursiveCharacterTextSplitter
# from langchain_text_splitters import TokenTextSplitter
# Alternative, wenn anstelle von Zeichen anhand von Token gesplittet werden soll 
# --> Realistischere Token-Grenzen durch Splitting möglich
# splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=100)
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

from termcolor import colored


# Vektordatenbank aus Bedienungsanleitungen erzeugen

In [ ]:
# Load the manuals
docs = []
manuals_path = 'manuals/'
for path in [f"{manuals_path}stellarwave_a9_manual.md", f"{manuals_path}luminor_arc65_manual.md"]:
    loader = TextLoader(path, encoding='utf-8')
    docs.extend(loader.load())

# Dokumente in einzelne Abschnitte unterteilen
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs_split = splitter.split_documents(docs)

# Embeddings erzeugen, spezieller Embedding-Encoder (ACHTUNG: OpenAI-API)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Speicherung in einer FAISS Vektordatenbank
vectorstore = FAISS.from_documents(docs_split, embeddings)
vectorstore.save_local("vector_db_manuals")

print("Vector DB created and saved to ./vector_db_manuals")


Vector DB created and saved to ./vector_db_manuals


## Beispielhafte Abfrage der Vektordatenbank

In [9]:
# Beispielabfrage 
query = "How can I install my TV?"

print(colored(f"Anfrage: {query}", "cyan", attrs=["bold"]))

# Suche in Vektordb
results = vectorstore.similarity_search_with_score(query, k=2)  # k = Anzahl Treffer

# Ausgabe der Treffer
for i, (doc, score) in enumerate(results, start=1):
    print(colored(f"Treffer {i}: (Score: {score:.2f})", "green"))
    print(colored("------------------------", "green"))
    print(doc.page_content.strip())
    if 'source' in doc.metadata:
        print(f"--> (Quelle: {doc.metadata['source']})")
    print()


Anfrage: How can I install my TV?
Treffer 1: (Score: 1.07)
------------------------
### 3.2 Mounting Instructions
1. Remove the stand before wall installation.  
2. Align holes with **VESA 400 × 300 mm** bracket.  
3. Use **M6 × 16 mm** screws.  
4. Route cables through the provided rear channels for clean setup.

### 3.3 Placement Tips
- For fireplace setups: use a tilting bracket to maintain viewing angle below 15°.  
- For best color uniformity, view from at least **1.2× screen height** distance.
--> (Quelle: manuals/luminor_arc65_manual.md)

Treffer 2: (Score: 1.30)
------------------------
### 3.2 Mounting
The A9 supports both **desk stand** and **VESA 100 × 100 mm** wall mounts.  
For wall installation:
1. Remove the stand using a Phillips screwdriver.  
2. Align the bracket holes.  
3. Use **M4 × 10 mm** screws (not included).  
4. Maintain at least **10 cm clearance** around the rear vents.
--> (Quelle: manuals/stellarwave_a9_manual.md)



# Übergabe der Informationen an ein LLM mit RAG

In [ ]:
# Minimaler RAG-Flow 

# Datenbankzugriff auf Vektordatenbank vorbereiten
vectorstore = FAISS.load_local("vector_db_manuals", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Prompt-Vorlage
prompt = ChatPromptTemplate.from_template(
    """You are a helpful TV/monitor support assistant.
Use ONLY the CONTEXT to answer. If it's not in the manuals, 
say so by answering 'NO IMMEDIATE SOLUTION POSSIBLE' only!

QUESTION:
{question}

CONTEXT:
{context}

Answer in English. If steps are needed, list them."""
)

# Erzeuge einen Ablauf (chain):
# Prompt erzeugen --> LLM aufrufen --> Ausgabe als String
llm = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()
chain = prompt | llm | parser

# Hilfsfunktion: Docs in einen String verwandeln
def make_context(docs):
    return "\n\n".join(f"[{i+1}] {d.page_content.strip()}" for i, d in enumerate(docs))

# „ask“-Funktion
def ask(question: str) -> str:
    docs = retriever.invoke(question)
    ctx = make_context(docs)
    
    full_prompt = prompt.format(question=question, context=ctx)
    print(colored("---- PROMPT SENT TO LLM ----", "cyan",  attrs=["bold"]))
    print(colored(full_prompt, "cyan"))
    print(colored("-----------------------------", "cyan", attrs=["bold"]))
    return chain.invoke({"question": question, "context": ctx})

# Beispiel-Ticket (ACHTUNG: Nutzung der OpenAI-API! Kosten entstehen)
# question = "My StellarWave A9 screen flickers on DisplayPort. What can I do?"
# question = "My StellarWave A9 is lying on the carpet. What can I do?"
question = "How can I mount my TV?"

print(colored(ask(question), "green"))


---- PROMPT SENT TO LLM ----
Human: You are a helpful TV/monitor support assistant.
Use ONLY the CONTEXT to answer. If it's not in the manuals, 
say so by answering 'NO IMMEDIATE SOLUTION POSSIBLE' only!

QUESTION:
How can I mount my TV?

CONTEXT:
[1] ### 3.2 Mounting Instructions
1. Remove the stand before wall installation.  
2. Align holes with **VESA 400 × 300 mm** bracket.  
3. Use **M6 × 16 mm** screws.  
4. Route cables through the provided rear channels for clean setup.

### 3.3 Placement Tips
- For fireplace setups: use a tilting bracket to maintain viewing angle below 15°.  
- For best color uniformity, view from at least **1.2× screen height** distance.

[2] ### 3.2 Mounting
The A9 supports both **desk stand** and **VESA 100 × 100 mm** wall mounts.  
For wall installation:
1. Remove the stand using a Phillips screwdriver.  
2. Align the bracket holes.  
3. Use **M4 × 10 mm** screws (not included).  
4. Maintain at least **10 cm clearance** around the rear vents.

[3] ## 2. S

# Evaluation der Antwort (hybrid: Vektorsuche + LLM as a Judge)
Ansatz: Sowohl die "Nähe" in der Vektordatenbank-Suche ermitteln, als auch die Antwort des LLMs durch ein weiteres LLM (LLM as a Judge) beurteilen lassen. 

In [5]:
def retrieve_from_vector_db_with_scores(query: str, k: int = 3):
    """ Liefert die Nähe der ermittelten Dokumente zur Query. Je kleiner der Score desto besser. """
    docs_scores = vectorstore.similarity_search_with_score(query, k=k)  # List[ (Document, score) ]
    return docs_scores

def normalize_faiss_scores(scores):
    """ normalisiert die Scores der Dokumente (zwischen 0 und 1), um Vergleichbarkeit herzustellen"""
    mn, mx = min(scores), max(scores)
    if mx == mn:
        return [1.0 for _ in scores]  # alle gleich gut -> max confidence
    sims = [1.0 - (s - mn) / (mx - mn) for s in scores]  # größer = besser
    return sims

# LLM als "Judge" anlegen
class EvalSchema(BaseModel):
    solution: str = Field(..., description="Proposed solution text to user")
    fix_likelihood: float = Field(..., ge=0.0, le=1.0,
        description="Model's estimate [0..1] that the solution will resolve the issue")
    rationale: str = Field(..., description="One or two sentences explaining why")

judge_prompt = ChatPromptTemplate.from_template(
    """You are a cautious support QA checker.
Use ONLY the CONTEXT to craft the solution. If the manuals don't contain the answer,
set solution to 'NO IMMEDIATE SOLUTION POSSIBLE' and fix_likelihood to 0.0.

QUESTION:
{question}

CONTEXT:
{context}

Return JSON with keys: solution, fix_likelihood, rationale.
Keep rationale concise (≤2 sentences)."""
)

judge_parser = JsonOutputParser(pydantic_object=EvalSchema)

# Verwendung des bestehenden LLMs für die Query
judge_chain = judge_prompt | llm | judge_parser 

def ask_with_confidence(question: str):
    # Vektordatenbank abfragen (inklusive Scores)
    docs_scores = retrieve_from_vector_db_with_scores(question, k=3)
    docs = [d for d, _ in docs_scores]
    scores = [s for _, s in docs_scores]
    sims = normalize_faiss_scores(scores)           # Normalisierung auf [0..1]
    retrieval_conf = sum(sims) / max(1, len(sims))  # Durchschnitt der Scores

    # Kontext bauen
    ctx = "\n\n".join(f"[{i+1}] {d.page_content.strip()}" for i, d in enumerate(docs))

    # Rufe Judge LLM auf
    result = judge_chain.invoke({"question": question, "context": ctx})
    solution = result["solution"]  # Modell liefert strukturierte JSON-Daten!
    model_conf = float(result["fix_likelihood"])  # Modell liefert strukturierte JSON-Daten!

    # Gewichtete Mischung aus der Vektor-DB-Konfidenz und der des Judge-Modells
    w_retrieval, w_model = 0.6, 0.4  # 60% aud Vektor-DB zu 40% aus Judge-Modell
    final_conf = max(0.0, min(1.0, w_retrieval * retrieval_conf + w_model * model_conf))

    return {
        "answer": solution,
        "confidence": round(final_conf, 2),
        "retrieval_conf": round(retrieval_conf, 2),
        "model_conf": round(model_conf, 2),
        "rationale": result["rationale"],
        "sources": [d.metadata.get("source", "manual_chunk") for d in docs]
    }

# Beispiel:
# question = "My StellarWave A9 screen flickers on DisplayPort. What can I do?"
# question = "My StellarWave A9 is lying on the carpet. What can I do?"
question = "How can I mount my TV?"

res = ask_with_confidence(question)
print(colored(f"Question: {question}", "cyan", attrs=["bold"]))
print(colored(f"Confidence (final): {res['confidence']:.2f} (retrieval {res['retrieval_conf']:.2f}, judge model {res['model_conf']:.2f})", "green", attrs=["bold"]))
print(colored(f"Rationale: {res['rationale']}", "yellow"))


Question: How can I mount my TV?
Confidence (final): 0.71 (retrieval 0.51, judge model 1.00)
Rationale: The provided context outlines clear mounting instructions, specifying tool requirements and safety considerations.
